In [1]:
import gpt as g
import matplotlib.pyplot as plt
import numpy as np

## Lecture 3: A first look at the QCD module.  We generate a quenched ensemble, measure the wilson flow scale and compute a domain-wall pion correlator.

We start by creating a $8^3 \times 32$ double precision grid.

In [2]:
grid = g.grid([8, 8, 8, 32], g.double)

Next, we create a parallel pseudorandom number generator and a unit gauge configuration.

In [3]:
rng = g.random("test")
U = g.qcd.gauge.unit(grid)

GPT :       1.989981 s : Initializing gpt.random(test,vectorized_ranlux24_389_64) took 0.000581026 s


The gauge field is a list of color matrices, currently initialized to unit matrices.

In [4]:
Nd = len(U)
g.message(f"We live in {Nd} space-time dimensions")
g.message(f"U[0][0,0,0,0] = {U[0][0,0,0,0]}")

GPT :       2.322732 s : We live in 4 space-time dimensions
GPT :       2.357127 s : U[0][0,0,0,0] = tensor([[1.+0.j 0.+0.j 0.+0.j]
                       :  [0.+0.j 1.+0.j 0.+0.j]
                       :  [0.+0.j 0.+0.j 1.+0.j]],ot_matrix_su_n_fundamental_group(3))


Next, we will prepare for applying a SU$(2)$ subgroup heatbath algorithm to generate an ensemble of quenched QCD configurations.  A reasonable updating scheme, first updates the even and then the odd sites.  So we need helper fields to mask these subsets.

In [5]:
grid_eo = grid.checkerboarded(g.redblack)
mask_rb = g.complex(grid_eo)
mask_rb[:] = 1
mask = g.complex(grid)

Let us now generate a pure Wilson gauge ensemble with $\beta=5.5$.  For this, we first define a staple and the parameter.

In [6]:
# gauge action parameter
beta = 5.5

# simple plaquette action
def staple(U, mu):
    st = g.lattice(U[0])
    st[:] = 0
    Nd = len(U)
    for nu in range(Nd):
        if mu != nu:
            st += beta * g.qcd.gauge.staple(U, mu, nu) / U[0].otype.Nc
    return st

In [ ]:
g.default.push_verbose("su2_heat_bath", False) # disable verbose algorithm

markov = g.algorithms.markov.su2_heat_bath(rng)

for it in range(100):
    plaq = g.qcd.gauge.plaquette(U)
    R_2x1 = g.qcd.gauge.rectangle(U, 2, 1)
    g.message(f"SU(2)-subgroup heatbath {it} has P = {plaq}, R_2x1 = {R_2x1}")
    for cb in [g.even, g.odd]:
        mask[:] = 0
        mask_rb.checkerboard(cb)
        g.set_checkerboard(mask, mask_rb)

        for mu in range(Nd):
            st = g.eval(staple(U, mu))
            markov(U[mu], st, mask)

GPT :       5.491279 s : SU(2)-subgroup heatbath 0 has P = 1.0, R_2x1 = 1.0
GPT :       7.387861 s : SU(2)-subgroup heatbath 1 has P = 0.6690106656133704, R_2x1 = 0.49840042565202647
GPT :       9.023182 s : SU(2)-subgroup heatbath 2 has P = 0.5975299501685577, R_2x1 = 0.3989602320643261
GPT :      10.746367 s : SU(2)-subgroup heatbath 3 has P = 0.5740740091875974, R_2x1 = 0.3651939333922665
GPT :      12.477127 s : SU(2)-subgroup heatbath 4 has P = 0.5602114116790958, R_2x1 = 0.3453226269277516
GPT :      14.121398 s : SU(2)-subgroup heatbath 5 has P = 0.5512347885669556, R_2x1 = 0.3330227490132815
GPT :      15.788057 s : SU(2)-subgroup heatbath 6 has P = 0.544845988665109, R_2x1 = 0.3246753117627473
GPT :      17.507280 s : SU(2)-subgroup heatbath 7 has P = 0.5414547577156114, R_2x1 = 0.3204750536610094
GPT :      19.241020 s : SU(2)-subgroup heatbath 8 has P = 0.5389550422162215, R_2x1 = 0.31702852079061467
GPT :      20.914516 s : SU(2)-subgroup heatbath 9 has P = 0.53740287823156

GPT :     140.229697 s : SU(2)-subgroup heatbath 77 has P = 0.49978392775493624, R_2x1 = 0.2633221720791911


We can now save the current gauge configuration to a file.

In [ ]:
g.save("/notebooks/ckpoint.lat", U, g.format.nersc())

For completeness, here is how we would load a gauge configuration:

In [ ]:
Uprime = g.load("/notebooks/ckpoint.lat")

Next, let us smear the gauge field using the Wilson flow and a fourth-order Runge-Kutta scheme:

In [ ]:
t = 0.0
eps = 0.05
U_wf = g.copy(U)
plot_x = []
plot_y = []
for i in range(15):
    U_wf = g.qcd.gauge.smear.wilson_flow(U_wf, epsilon=eps)
    t += eps
    E = g.qcd.gauge.energy_density(U_wf)
    g.message(f"t^2 E(t={t:g})={t**2. * E}")
    plot_x.append(t)
    plot_y.append(t**2. * E)

We can use this to approximately set a scale for this ensemble:

In [ ]:
plt.plot(plot_x, plot_y, 'ro')
plt.show()

t0_over_asqr = 0.525
ainvOverGeV = 1.3 * t0_over_asqr**0.5  # live in world with t0^-1/2 = 1.3 GeV
g.message(f"Lattice scale is approximately: {ainvOverGeV:.2g} GeV")

Now, let us prepare a mobius domain-wall fermion on this gauge configuration.

In [ ]:
qm = g.qcd.fermion.mobius(U, {
    "mass": 0.12,
    "M5": 1.8,
    "b": 1.5,
    "c": 0.5,
    "Ls": 12,
    "boundary_phases": [1.0, 1.0, 1.0, -1.0],
})

Create a physical four-dimensional propagator using a simple even-odd preconditioned solver.

In [ ]:
pc = g.qcd.fermion.preconditioner
inv = g.algorithms.inverter

g.default.push_verbose("cg_convergence", True) # want to see CG progress
Q = qm.propagator(inv.preconditioned(pc.eo2_ne(), inv.cg({"eps": 1e-6, "maxiter": 100}))).grouped(1)

Finally, let us create a point source at the origin and create a propagator field.

In [ ]:
src = g.mspincolor(grid)
g.create.point(src, [0, 1, 0, 0])

prop_field = g.eval( Q * src )

In [ ]:
corr = g.slice(g.trace(prop_field * g.adj(prop_field)),3)

In [ ]:
E_mass = [np.log(corr[t]/corr[t+1]).real for t in range(16)]

In [ ]:
plt.plot(range(16), E_mass, 'ro')
plt.ylim(0,2)
plt.show()